In [2]:
from keras.applications.xception import Xception
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras import applications
from keras import optimizers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

import numpy as np
import argparse

In [3]:
base_model = applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape = (224,224,3))
for layer in base_model.layers:
    layer.trainable = False
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [4]:
# for layer in base_model.layers:
#     print(layer.trainable)

In [7]:
x = base_model.output
x = Flatten()(x)
x = Dense(256)(x)
x = Dense(256)(x)
# x = Dropout(0.5)(x)
#predictions = Dense(2, activation="softmax") (x)
#predictions = Dense(1, activation="sigmoid") (x)
x = Dense(10, activation='softmax', name='predictions')(x)


model_final = Model(inputs=base_model.input, outputs=x)
model_final.compile(loss="categorical_crossentropy", optimizer=optimizers.sgd(lr=0.0001, momentum=0.9), metrics=["accuracy"])

model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
train_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)

train_data_dir = 'data/train'
validation_data_dir = "data/validation"
nb_validation_samples = 1000
nb_train_samples = 5000
epochs = 5
batch_size = 16
img_width, img_height = 224, 224

train_generator = train_datagen.flow_from_directory(
                train_data_dir,
                target_size=(img_width, img_height),
                batch_size=batch_size,
                class_mode='categorical',
                shuffle=True)

val_generator = val_datagen.flow_from_directory(
                                validation_data_dir,
                                target_size = (img_height, img_width),
                                batch_size=batch_size,
                                class_mode = "categorical")

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples//batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=nb_validation_samples//batch_size,
        verbose=1,
        workers=4,
        callbacks = [checkpoint, early])

Found 5000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Epoch 1/5
  1/312 [..............................] - ETA: 2654s - loss: 3.5198 - acc: 0.0000e+00

KeyboardInterrupt: 

In [31]:
print('abc')

abc


In [ ]:
model.save_weights('vgg16_classifier_first_try.h5')